### Initializing Arduino relay, Keithley current source, and coil parameters

##### NOTE: Running the "initialize" cell more than once per kernel session will throw errors since the resources are already accessed

In [10]:
arduinoport = "COM4"
keithleyname = "ASRL5::INSTR"

In [11]:
import serial
import time
import sys
import math as np
import pyvisa as visa
# visa.log_to_screen()
rm = visa.ResourceManager()
import pyfirmata


In [12]:
def initializerelay(port):
    board = pyfirmata.Arduino(port)
    it = pyfirmata.util.Iterator(board)
    it.start()
    return board
arduino = initializerelay(arduinoport)


def initializecoil(port):

    keithley = rm.open_resource(port)
    keithley.read_terloweration = "\n"
    keithley.write_termination = '\n'
    print(keithley.query("*IDN?"))
    keithley.write("SYST:REM")
    keithley.write("OUTP:STAT:ALL OFF")
    keithley.write('APP:VOLT 3.000000,3.000000,3.000000')
    keithley.write("APP:CURR 0.000000,0.000000,0.000000")
    print("Keithley Initialized")
    return keithley

keithley = initializecoil(keithleyname)


fieldcoeffs = {
    'X' : [1.36932, -0.0174511], ##slope, intercept
    'Y' : [2.13605, 0.0259495],
    'Z' : [1.24437, -0.0123547],
}

Keithley instruments, 2231A-30-3, 802196010727610334, 1.04-1.04

Keithley Initialized


In [18]:
keithley.close()
arduino.exit()

### Control functions for Arduino and Keithley

In [13]:
###Keithley controls
def querychannel(channel, type):
    keithley.write("INST:NSEL " + str(channel))
    dat = {}
    if type == "ALL":
        dat['channel'] = keithley.query("INST?")
        dat['CURR'] = keithley.query("MEAS:CURR?")
        dat['VOLT'] = keithley.query("MEAS:VOLT?")
    else: 
        dat['channel'] = keithley.query("INST?")
        dat[type] = keithley.query("MEAS:" + type + "?")
    return dat
    
# def write3channels(TYPE, x, y, z):
#     "Writes the TYPE (CURR or VOLT) to all three channels x, y, z on the current source"
#     s = "APP:" + TYPE + " " + str(abs(x)) + ","+str(abs(y))+"," + str(abs(z))
#     keithley.write(s)
#     return s


def query3channels(type):
    dat = {}
    dat["X"] = querychannel(1, type)
    dat["Y"] = querychannel(2, type)
    dat["Z"] = querychannel(3, type)
    return dat
def write3channels(inst, TYPE, x, y, z):
    "Writes the TYPE (CURR or VOLT) to all three channels x, y, z on the current source"
    s = "APP:" + TYPE + " " + str(abs(x)) + ","+str(abs(y))+"," + str(abs(z))
    inst.write(s)
    return s


def fieldtocurrent(B, phi, theta, fieldcoeffs, lower, upper):
    "Converts the Bnorm, phi, and theta input values into x, y, z, current values using calibration parameters in fieldcoeffs"
    x, errx = calibbfieldtocurr(fieldcoeffs['X'], B * np.sin(theta)* np.cos(phi), lower, upper)
    y, erry = calibbfieldtocurr(fieldcoeffs['Y'], B * np.sin(theta) * np.sin(phi), lower, upper)
    z, errz = calibbfieldtocurr(fieldcoeffs['Z'], B * np.cos(theta), lower, upper)
    if errx==erry==errz==None: return x,y,z, None
    else: return x,y,z, [errx, erry, errz]


def calibbfieldtocurr(fieldcoeffs, x, lower, upper):
    "Calibrates the field to current conversion using parameters in fieldcoeffs"
    y = (x - fieldcoeffs[1])/fieldcoeffs[0]
    if lower<=y<=upper:
        return y, None
    elif y>upper: return upper, "Current clipped to " + str(upper) + "A"
    elif y<lower: return lower, "Current clipped to " + str(lower) + "A"


def setpolarity(x,y,z):
    "Determines the 3-bit polarity string to send to the Arduino Relay. 0 for positive polarity, 1 for negative polarity"
    pols = []
    if abs(x) == x: pols.append(0)
    else: pols.append(1)

    if abs(y) == y: pols.append(0)
    else: pols.append(1)

    if abs(z) == z: pols.append(0)
    else: pols.append(1)
    pols = "".join(str(pol) for pol in pols)
    print(pols)
    setbfieldpol(pols)
    output, d = getbfieldpol()

    return d, pols


def currenttofield(fieldcoeffs, x1, y1, z1):
    "Converts the x,y,z current input values into bnorm, phi, and theta values using calibration parameters in fieldcoeffs"

    x = calibcurrtobfield(fieldcoeffs["X"], x1)
    y = calibcurrtobfield(fieldcoeffs["Y"], y1)
    z = calibcurrtobfield(fieldcoeffs["Z"], z1)

    bnorm = np.sqrt(x**2 + y**2 + z**2)
    phi = np.atan(y/x)
    theta = np.acos(z/bnorm)
    return bnorm, phi, theta


def calibcurrtobfield(fieldcoeffs, x):
    "Calibrates the current to field conversion using parameters in fieldcoeffs"
    return (x * fieldcoeffs[0]) + fieldcoeffs[1]
    

def keithleyoff():
    keithley.write("OUTP:STAT:ALL OFF")
    if keithley.query("OUTP:STAT:ALL?")== '0\n':
        print("Keithley outputs disabled")
    else: print("Error, outputs may not be disabled")


###Arduino controls
def setbfieldpol(pols): ####neg = 1, pos = 0
    "Writes the polarity to the Arduino relay. Must be a 3-bit string made of 0s and 1s"
    settings = "set " + pols + '\n'
    arduino.write(bytes(settings, 'utf-8'))
    time.sleep(.1)
    d = arduino.readline()
    arduino.flush()
    arduino.sendBreak()
    arduino.flush()
    return d


def getbfieldpol():
    "Reads the field polarities from the Arduino relay"
    settings = 'get \n'
    arduino.write(bytes(settings, 'utf-8'))
    time.sleep(.1)
    output = {}
    pols = arduino.readline()
    print(pols)
    

    if pols[0] == "0": x = 1
    else: x = -1
    if pols[1] == "0": y = 1
    else: y = -1
    if pols[2] == "0": z = 1
    else: z = -1
    print(x,y,z)

    arduino.flush()
    arduino.sendBreak()
    arduino.flush()    
    return output, pols


def setfield(bnorm, phi, theta, magneton):
    if magneton == True:
        keithley.write("OUTP:STAT:ALL ON")

    x,y,z, errs = fieldtocurrent(bnorm, phi, theta, fieldcoeffs, -3., 3.) ## Step 1 Convert input B field values to current values
    print("Any clipped currents? ", errs)

    d,pols = setpolarity(x,y,z) ##Step 2 set the Arduino relay polarities
    print("String sent to Arduino relay and string read from Arduino: ", d, pols) ###d and pols should be the same (pols is what you're trying to set, d is what is read out)


    s = write3channels(keithley, "CURR", x,y,z) ##Step 3 sets the current (using absolute values since this is setting to the Keithley and the Arduino will deal with the polarities)
    print("String sent to kethiley: ", s)
    dat = query3channels(keithley)
    print("Final readings from Keithley: ", dat)
    xf = float(dat["X"]["current"])
    yf = float(dat["Y"]["current"])
    zf = float(dat["Z"]["current"])

    bnormf, phif, thetaf = currenttofield(fieldcoeffs, xf, yf, zf) ##Convert read values to field values
    print("Field set. Magnetic field readings:")
    
    print("Set Bnorm", bnorm, "\t", "Read Bnorm", bnormf)
    print("Set phi", phi, "\t", "Read phi", phif)
    print("Set theta", theta, "\t", "Read theta", thetaf)


### Run cells up to here to initialize magnet system. Change parameters below before moving on.

In [5]:
bnorm = 0.5 ## in mT
phi = 0.7
theta = 1.57
magneton = True


In [ ]:
arduino.write(bytes("set 111", "utf-8"))

In [ ]:
arduino.readline()

### Run to set field

In [10]:
setfield(bnorm, phi, theta, magneton)

Any clipped currents?  None
000


AttributeError: 'Arduino' object has no attribute 'write'

In [15]:
keithley.write("OUTP:STAT:ALL ON")

17

In [16]:
keithley.query("MEAS:CURR?")

'0.00459881\n'

### TURN OFF KEITHLEY OUTPUTS AFTER MEASUREMENTS

In [ ]:
import numpy as np
def convertpols(pols):
    d = int(pols)
    op1 = np.floor(d/100)
    print(op1)
    if op1 == 0: x = 1
    else: x = -1
    op2 = np.remainder(d, 100)
    op2 = np.floor(op2/10)
    print(op2)
    if np.floor(op2) == 0: y = 1
    else: y = -1
    op3 = np.remainder(d,10)
    print(op3)
    if np.floor(op3) == 0: z = 1
    else: z = -1
    print(x,y,z)
    return x, y, z


In [ ]:
convertpols("011")

In [ ]:
d = 101
op1= np.divmod(d, 100)
op2 = np.divmod(d, 10)
op3 = np.divmod(d, 1)


In [ ]:
print(op1, op2, op3)

In [2]:
import pyfirmata
import time

In [3]:
board = pyfirmata.Arduino("COM4")
it = pyfirmata.util.Iterator(board)
it.start()

In [5]:
def controlrelay(input):
    s = input.split()
    mode = s[0]
    arg = s[1]
    if mode == "set":
        setrelay(arg)
        read = getrelay()
    else: 
        read = getrelay
    
    return mode, arg, read

def polarizationflip(pol, channel1, channel2):
    if pol=="1":
        board.digital[channel1].write(1)
        board.digital[channel2].write(1)
    else: 
        board.digital[channel1].write(0)
        board.digital[channel2].write(0)

def setrelay(pols):
    polarizationflip(pols[0], 7, 8)
    polarizationflip(pols[1], 9, 10)
    polarizationflip(pols[2], 11, 12)


def getrelay():
    pins = [7,8,9,10,11,12]
    read = []
    for i in pins:
        read.append(board.digital[i].read())
    return read

In [20]:
board.exit()

In [7]:
controlrelay("set 111")

('set', '111', [1, 1, 1, 1, 1, 1])